<a id='Data_Preprocessing'></a>

# Restaurant Fake Review Detection # 1

- __[Preparation](#Preparation)__
- __[Data Cleaning](#Data_Cleaning)__

<a id='Preparation'></a>

# Preparation

In [16]:
# Import libraries required for analysis

import sqlite3
import pandas as pd
import numpy as np
from operator import attrgetter
import re
import string
from textblob import TextBlob

In [17]:
# Prepare to extract dataset from SQLite database
dbfile = 'yelpResData.db'
con = sqlite3.connect(dbfile)
con.text_factory = lambda b: b.decode(errors = 'ignore')
cur = con.cursor()

# Check the tables in the database
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)

[('review',), ('restaurant',), ('reviewer',)]


There are three tables in the database; review, restaurant, and reviewer. 

The following are the column names in each of the tables. The column names are self-explanatory. Please see the __codebook ("Yelp_Restaurant_Codebook")__ that explains what each column represents. The restaurants in this dataset are in Chicago, Illinois. 

In [18]:
cursor_review = con.execute('select * from review')
cursor_restaurant = con.execute('select * from restaurant')
cursor_reviewer = con.execute('select * from reviewer')

review_colNames = list(map(lambda x: x[0], cursor_review.description))
restaurant_colNames = list(map(lambda x: x[0], cursor_restaurant.description))
reviewer_colNames = list(map(lambda x: x[0], cursor_reviewer.description))

In [19]:
print("review :", review_colNames)
print("restaurant: ", restaurant_colNames)
print("reviewer: ", reviewer_colNames)

review : ['date', 'reviewID', 'reviewerID', 'reviewContent', 'rating', 'usefulCount', 'coolCount', 'funnyCount', 'flagged', 'restaurantID']
restaurant:  ['restaurantID', 'name', 'location', 'reviewCount', 'rating', 'categories', 'address', 'Hours', 'GoodforKids', 'AcceptsCreditCards', 'Parking', 'Attire', 'GoodforGroups', 'PriceRange', 'TakesReservations', 'Delivery', 'Takeout', 'WaiterService', 'OutdoorSeating', 'WiFi', 'GoodFor', 'Alcohol', 'NoiseLevel', 'Ambience', 'HasTV', 'Caters', 'WheelchairAccessible', 'webSite', 'phoneNumber', 'filReviewCount']
reviewer:  ['reviewerID', 'name', 'location', 'yelpJoinDate', 'friendCount', 'reviewCount', 'firstCount', 'usefulCount', 'coolCount', 'funnyCount', 'complimentCount', 'tipCount', 'fanCount']


The 'review' table contains restaurantID and reviewerID. Using the ID's, we will join all three tables later. However, the ID's and name columns will be dropped to protect personal/private information. And some of the columns which are not useful when building detection models will also be dropped. 

In [20]:
# Join the 'review', 'restaurant', and 'reviewer' tables

sql_query_join = """
SELECT  a.date,
        a.reviewContent,
        a.rating,
        a.flagged,
        a.reviewerID,
        b.location AS reviewer_location,
        b.yelpJoinDate,
        b.friendCount AS reviewer_friendCount,
        b.reviewCount AS reviewer_reviewCount,
        b.firstCount AS reviewer_firstCount,
        b.usefulCount AS reviewer_usefulCount,
        b.coolCount AS reviewer_coolCount,
        b.funnyCount AS reviewer_funnyCount,
        b.complimentCount AS reviewer_complimentCount,
        b.tipCount AS reviewer_tipCount,
        b.fanCount AS reviewer_fanCount,
        c.reviewCount AS restaurant_reviewCount,
        c.rating AS restaurant_rating, 
        c.categories,
        c.PriceRange
        
FROM review a
INNER JOIN reviewer b ON a.reviewerID = b.reviewerID
LEFT JOIN restaurant c ON a.restaurantID = c.restaurantID
WHERE a.flagged in ('Y', 'N')
"""

df_join = pd.read_sql_query(sql_query_join, con)

In [21]:
df_join.loc[:, df_join.columns != 'reviewerID']

,date,reviewContent,rating,flagged,reviewer_location,yelpJoinDate,reviewer_friendCount,reviewer_reviewCount,reviewer_firstCount,reviewer_usefulCount,reviewer_coolCount,reviewer_funnyCount,reviewer_complimentCount,reviewer_tipCount,reviewer_fanCount,restaurant_reviewCount,restaurant_rating,categories,PriceRange
0,9/22/2012,"Unlike Next, which we'd eaten at the previous ...",5,N,"Glengary, WV",October 2009,11,48,5,41,5,5,2,0,1,841,4.5,"Restaurants, American (New)",$$$$
1,9/23/2012,The Purple Pig was one of the restaurants I mo...,4,N,"Glengary, WV",October 2009,11,48,5,41,5,5,2,0,1,1312,4.0,"Restaurants, Modern European",$$
2,9/22/2012,Probably one of the best meals I've had ever. ...,5,N,"Palos Verdes Peninsula, CA",June 2011,0,4,0,0,0,0,0,0,0,841,4.5,"Restaurants, American (New)",$$$$
3,9/19/2012,Service was impeccable. Experience and present...,3,N,"Chicago, IL",September 2008,2,11,0,6,0,1,0,0,0,841,4.5,"Restaurants, American (New)",$$$$
4,9/6/2012,"The problem with places like this, given the e...",3,N,"San Jose, CA",October 2007,84,60,1,35,8,6,6,11,0,841,4.5,"Restaurants, American (New)",$$$$
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26953,8/11/2011,I made my way to The Publican on August 10th f...,2,N,"Chicago, IL",September 2010,15,4,0,14,3,0,0,1,0,1303,4.0,"Nightlife, Bars, Pubs, Restaurants, American (...",$$$
26954,8/31/2011,You don't get much better than this. Reasonab...,5,N,"Chicago, IL",June 2009,0,10,0,5,0,1,0,0,0,1303,4.0,"Nightlife, Bars, Pubs, Restaurants, American (...",$$$
26955,5/23/2011,The food was good but for the money you spend ...,3,N,"Stoughton, WI",November 2009,3,147,16,47,10,18,2,0,1,1303,4.0,"Nightlife, Bars, Pubs, Restaurants, American (...",$$$
26956,5/23/2011,"If you like deep dish/Chicago style pizza, thi...",3,N,"Stoughton, WI",November 2009,3,147,16,47,10,18,2,0,1,701,3.5,"Restaurants, Pizza",$$


<a id='Data_Cleaning'></a>

# Data Cleaning

In [22]:
# Check the number of missing values in each column
print(df_join.isnull().sum())

# Drop rows with NaN (PriceRange contains 2 NaN's)
df_join = df_join.dropna()

date                        0
reviewContent               0
rating                      0
flagged                     0
reviewerID                  0
reviewer_location           0
yelpJoinDate                0
reviewer_friendCount        0
reviewer_reviewCount        0
reviewer_firstCount         0
reviewer_usefulCount        0
reviewer_coolCount          0
reviewer_funnyCount         0
reviewer_complimentCount    0
reviewer_tipCount           0
reviewer_fanCount           0
restaurant_reviewCount      0
restaurant_rating           0
categories                  0
PriceRange                  2
dtype: int64


In [23]:
# Rename levels of 'PriceRange'.

for i in range(df_join.shape[0]):
    if df_join.loc[df_join.index[i], 'PriceRange'] == '$':
        df_join.loc[df_join.index[i], 'PriceRange'] = 'Under $10'
    elif df_join.loc[df_join.index[i], 'PriceRange'] == '$$':
        df_join.loc[df_join.index[i], 'PriceRange'] = '$11-30'
    elif df_join.loc[df_join.index[i], 'PriceRange'] == '$$$':
        df_join.loc[df_join.index[i], 'PriceRange'] = '$31-60' 
    else:
        df_join.loc[df_join.index[i], 'PriceRange'] = 'Over $61'

In [24]:
# There are no duplicated rows in this dataset. 

print(df_join[df_join.duplicated()])

Empty DataFrame
Columns: [date, reviewContent, rating, flagged, reviewerID, reviewer_location, yelpJoinDate, reviewer_friendCount, reviewer_reviewCount, reviewer_firstCount, reviewer_usefulCount, reviewer_coolCount, reviewer_funnyCount, reviewer_complimentCount, reviewer_tipCount, reviewer_fanCount, restaurant_reviewCount, restaurant_rating, categories, PriceRange]
Index: []


In [25]:
# However, a reviewer left the same review contents at different restaurants

dup_ID = df_join[df_join[['reviewContent', 'reviewerID']].duplicated()].reviewerID.item()
df_join.loc[df_join['reviewerID'] == dup_ID, df_join.columns != 'reviewerID']

,date,reviewContent,rating,flagged,reviewer_location,yelpJoinDate,reviewer_friendCount,reviewer_reviewCount,reviewer_firstCount,reviewer_usefulCount,reviewer_coolCount,reviewer_funnyCount,reviewer_complimentCount,reviewer_tipCount,reviewer_fanCount,restaurant_reviewCount,restaurant_rating,categories,PriceRange
18947,3/14/2012,"Not sure why my review wasn't posted, but I li...",4,Y,"Chicago, IL",February 2012,4,5,0,3,0,0,1,0,0,1182,4.0,"Restaurants, American (New), Gastropubs",$11-30
18948,\n3/14/2012,"Not sure why my review wasn't posted, but I li...",4,Y,"Chicago, IL",February 2012,4,5,0,3,0,0,1,0,0,348,3.5,"Restaurants, Sushi Bars",$11-30


In [26]:
# Convert the date columns into date formats which are originally in string formats

df_join['date'] = pd.to_datetime(df_join['date'])
df_join['yelpJoinDate'] = pd.to_datetime(df_join['yelpJoinDate']).dt.to_period('M')

In [27]:
# Some categorical features can have redundant categories. We will lump in such levels into a new group called 'other'. 

# Find features with 10 or more levels. 
col_names = ['flagged', 'categories','PriceRange']
for col in col_names:
    if df_join[col].nunique() >= 10:
        print(col, df_join[col].nunique(), sep=' - count: ')
        

# Mask levels with low counts and call them 'Other'- Now there are 8 unique levels in 'categories'
df_join.loc[df_join['categories'].isin((df_join['categories'].value_counts()[df_join['categories'].value_counts() < 690]).index), 'categories'] = 'Other'
print('categories', df_join['categories'].nunique(), sep=' - count: ')

categories - count: 82
categories - count: 8


In [28]:
# The text data (reviewContent) has special letters, capital letters, etc. They need to be cleaned for efficient analysis.

def data_clean(col):
    col = col.lower()
    col = re.sub('[%s]' % re.escape(string.punctuation), '', col)
    col = re.sub('\w*\d\w*', '', col)
    col = re.sub('\xa0', '', col)
    return col

dat_clean = lambda x: data_clean(x)

df_join["reviewContent"] = pd.DataFrame(df_join.reviewContent.apply(dat_clean))

In [29]:
df_join.loc[:, df_join.columns != 'reviewerID']

,date,reviewContent,rating,flagged,reviewer_location,yelpJoinDate,reviewer_friendCount,reviewer_reviewCount,reviewer_firstCount,reviewer_usefulCount,reviewer_coolCount,reviewer_funnyCount,reviewer_complimentCount,reviewer_tipCount,reviewer_fanCount,restaurant_reviewCount,restaurant_rating,categories,PriceRange
0,2012-09-22,unlike next which wed eaten at the previous ni...,5,N,"Glengary, WV",2009-10,11,48,5,41,5,5,2,0,1,841,4.5,"Restaurants, American (New)",Over $61
1,2012-09-23,the purple pig was one of the restaurants i mo...,4,N,"Glengary, WV",2009-10,11,48,5,41,5,5,2,0,1,1312,4.0,"Restaurants, Modern European",$11-30
2,2012-09-22,probably one of the best meals ive had ever it...,5,N,"Palos Verdes Peninsula, CA",2011-06,0,4,0,0,0,0,0,0,0,841,4.5,"Restaurants, American (New)",Over $61
3,2012-09-19,service was impeccable experience and presenta...,3,N,"Chicago, IL",2008-09,2,11,0,6,0,1,0,0,0,841,4.5,"Restaurants, American (New)",Over $61
4,2012-09-06,the problem with places like this given the ex...,3,N,"San Jose, CA",2007-10,84,60,1,35,8,6,6,11,0,841,4.5,"Restaurants, American (New)",Over $61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26953,2011-08-11,i made my way to the publican on august for t...,2,N,"Chicago, IL",2010-09,15,4,0,14,3,0,0,1,0,1303,4.0,Other,$31-60
26954,2011-08-31,you dont get much better than this reasonable ...,5,N,"Chicago, IL",2009-06,0,10,0,5,0,1,0,0,0,1303,4.0,Other,$31-60
26955,2011-05-23,the food was good but for the money you spend ...,3,N,"Stoughton, WI",2009-11,3,147,16,47,10,18,2,0,1,1303,4.0,Other,$31-60
26956,2011-05-23,if you like deep dishchicago style pizza this ...,3,N,"Stoughton, WI",2009-11,3,147,16,47,10,18,2,0,1,701,3.5,"Restaurants, Pizza",$11-30


In [30]:
# Pickle the processed data for later use. 

df_join.to_pickle("./org_df.pkl")